# Step 1: Calling the API :~)

In [3]:
# STEP 1: Call an LLM using OpenRouter + DeepSeek

# 1. Ensure dependencies are installed via requirements.txt

import os
from openai import OpenAI
from dotenv import load_dotenv

# 2. Load API key from .env file
# Make sure your .env contains:
# OPENROUTER_API_KEY=<yourkey>
load_dotenv()

# 3. Initialise client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

# 4. Call the free DeepSeek model
response = client.chat.completions.create(
    model="deepseek/deepseek-chat-v3.1:free",   
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! Give me a fun fact about Singapore."}
    ]
)

# 5. Print the model’s response
print(response.choices[0].message.content)

Of course! Here's a fun fact:

Singapore is one of only three surviving city-states in the world. The other two are Monaco and Vatican City.

Despite its small size, it's a global hub for finance, trade, and transportation, making it a modern-day marvel of urban planning and economic success!


# Step 2: Generating SQL query by passing in db schema

In [6]:
schema = """{'schema': [{'cid': 0, 'name': 'year', 'type': 'INTEGER'},
  {'cid': 1, 'name': 'month_num', 'type': 'INTEGER'},
  {'cid': 2, 'name': 'town', 'type': 'VARCHAR'},
  {'cid': 3, 'name': 'flat_type', 'type': 'VARCHAR'},
  {'cid': 4, 'name': 'block', 'type': 'VARCHAR'},
  {'cid': 5, 'name': 'street_name', 'type': 'VARCHAR'},
  {'cid': 6, 'name': 'storey_range', 'type': 'VARCHAR'},
  {'cid': 7, 'name': 'floor_area_sqm', 'type': 'DOUBLE'},
  {'cid': 8, 'name': 'flat_model', 'type': 'VARCHAR'},
  {'cid': 9, 'name': 'lease_commence_date', 'type': 'INTEGER'},
  {'cid': 10, 'name': 'remaining_lease', 'type': 'VARCHAR'},
  {'cid': 11, 'name': 'resale_price', 'type': 'DOUBLE'}],
 'categories': {'flat_type': ['1 ROOM',
   '2 ROOM',
   '3 ROOM',
   '4 ROOM',
   '5 ROOM',
   'EXECUTIVE',
   'MULTI-GENERATION'],
  'town': ['ANG MO KIO',
   'BEDOK',
   'BISHAN',
   'BUKIT BATOK',
   'BUKIT MERAH',
   'BUKIT PANJANG',
   'BUKIT TIMAH',
   'CENTRAL AREA',
   'CHOA CHU KANG',
   'CLEMENTI',
   'GEYLANG',
   'HOUGANG',
   'JURONG EAST',
   'JURONG WEST',
   'KALLANG/WHAMPOA',
   'MARINE PARADE',
   'PASIR RIS',
   'PUNGGOL',
   'QUEENSTOWN',
   'SEMBAWANG',
   'SENGKANG',
   'SERANGOON',
   'TAMPINES',
   'TOA PAYOH',
   'WOODLANDS',
   'YISHUN'],
  'flat_model': ['2-room',
   '3Gen',
   'Adjoined flat',
   'Apartment',
   'DBSS',
   'Improved',
   'Improved-Maisonette',
   'Maisonette',
   'Model A',
   'Model A-Maisonette',
   'Model A2',
   'Multi Generation',
   'New Generation',
   'Premium Apartment',
   'Premium Apartment Loft',
   'Premium Maisonette',
   'Simplified',
   'Standard',
   'Terrace',
   'Type S1',
   'Type S2'],
  'storey_range': ['01 TO 03',
   '04 TO 06',
   '07 TO 09',
   '10 TO 12',
   '13 TO 15',
   '16 TO 18',
   '19 TO 21',
   '22 TO 24',
   '25 TO 27',
   '28 TO 30',
   '31 TO 33',
   '34 TO 36',
   '37 TO 39',
   '40 TO 42',
   '43 TO 45',
   '46 TO 48',
   '49 TO 51']}}"""

In [7]:
def main_agent(user_input: str):
    messages = [
        {"role": "system", "content": (
            f"""You are an agent that answers questions about HDB resale prices.

            This is the schema to the resale db: {schema}

            Based on the user's query, generate the required DuckDB query. ONLY REPLY WITH THE QUERY.
            """
        )},
        {"role": "user", "content": user_input}
    ]

    response = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3.1:free",
        messages=messages,
        temperature=0.2
    )

    msg = response.choices[0].message.content
    
    return msg

# Interactive mode
user_question = input("Enter your HDB resale question: ")
print(main_agent(user_question))

# Example quick test
# print(main_agent("How has average resale price changed over the years?"))


SELECT year, AVG(resale_price) AS avg_resale_price FROM resale GROUP BY year ORDER BY year;


# Step 3: Tool Time!
So far, our LLM could generate SQL text (Step 2).
But that’s not very useful on its own — we also want to:
* actually run the SQL query on our database, and
* then use the results to give insights back to the user.
This is where **🪄 tool usage 🪄** comes in!
We let the LLM *decide when to call a tool* — and we handle the execution in Python.

In [9]:
import json
import duckdb
import pandas as pd

# Path to the DuckDB database file
DB_PATH = "database/HDB_data.db"

#### Defining a tool for SQL execution

We start by **describing the tool** in a JSON schema.

* The tool is named `execute_sql_query`.
* It takes one argument: a SQL query string.
* The LLM can call this tool whenever it needs to run a query.

Think of this like an **extension to the system prompt you give the llm**. It can call this anytime like an **API Call** that we have to resolve for it.

In [10]:
# Define the tool (execute_sql_query) that the LLM can call
# Think of this as an addition to your system prompt that the LLM will receive
tool = [
    {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "Execute the DuckDB SQL query and return the results.",
            "parameters": {
                "type": "object",
                "properties": {
                    "sql_query": {
                        "type": "string",
                        "description": "The DuckDB SQL query to be executed."
                    }
                },
                "required": ["sql_query"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

#### Implementing the tool in Python

Now when the LLM calls the tool `execute_sql_query` that we give via json schema, we need to resolve that call by doing ~something~ on our end. For this use case, we want to execute the sql query, and we can do that via a Python function with the same name.

In [11]:
# Function that actually runs the SQL against DuckDB
def execute_sql_query(sql_query: str):
    if not sql_query:
        return {"error": "No SQL query provided"}
    try:
        conn = duckdb.connect(DB_PATH)
        c = conn.cursor()
        c.execute(sql_query)
        cols = [d[0] for d in c.description] if c.description else []
        rows = c.fetchmany(1000)  # cap results for safety
        conn.close()
    except Exception as e:
        return {"error": f"Error executing SQL query: {str(e)}"}
    
    return {"result_df": {"columns": cols, "rows": rows}}

#### Writing the agent loop

`main_agent`, ties it all together.

The steps are:

1. Pass the schema and user query into the model.
2. If the user just asks about the schema → answer directly.
3. Otherwise, the model generates SQL and calls the `execute_sql_query` tool.
4. We intercept that tool call, run it in Python, and send the results back.
5. Finally, the model produces insights based on the real data.

This loop continues until the model stops making tool calls and gives a final answer.

In [18]:
# Agent function that coordinates LLM + tool
def main_agent(user_input: str):
    messages = [
        {"role": "system", "content": (
            f"""You are an agent that answers questions about HDB resale prices.

            This is the schema to the resale db named `resale_data_2017_to_2025`: {schema}

            If the user's query is just about the schema, answer directly.
            Otherwise:
            1. Generate a SQL query.
            2. Call the `execute_sql_query` tool to run it.
            3. Give insights based on the query results. REMEMBER TO ALWAYSSS end with insights.
            
            DO NOT redisplay the raw table in markdown, 
            it will be shown separately in the app. JUST give the insights.
            """
        )},
        {"role": "user", "content": user_input}
    ]

    results = None  # to hold SQL results

    while True:
        # Ask the model what to do
        response = client.chat.completions.create(
            model="deepseek/deepseek-chat-v3.1:free",
            messages=messages,
            tools=tool,
            temperature=0.2
        )

        msg = response.choices[0].message
        messages.append(msg)

        # If the model called a tool, run it
        if msg.tool_calls:
            for call in msg.tool_calls:
                print(f"🤖 Model wants to call tool: {call.function.name}")
                args = json.loads(call.function.arguments)

                if call.function.name == "execute_sql_query":
                    sql_query = args.get("sql_query")
                    print(f"📜 SQL generated: {sql_query}")

                    results = execute_sql_query(sql_query)
                    if results and "result_df" in results:
                        print(f"✅ Query executed, got {len(results['result_df']['rows'])} rows")
                    else:
                        print(f"{results['error']}")

                    messages.append({
                        "role": "tool",
                        "tool_call_id": call.id,
                        "content": json.dumps(results)
                    })

        else:
            # No tool call, so this must be the final answer
            print("💡 Insights from the agent:")
            print(msg.content)
            if not msg.content:
                print("❌ No insights found. Asking agent again")

            if results and "result_df" in results:
                return {"insights": msg.content,
                        "result_df": results["result_df"]}
            else:
                return {"insights": msg.content,
                        "result_df": None}
            break


# Example usage
output = main_agent("How has average resale price changed over the years?")

# Print results nicely
print("\n=== Insights ===")
print(output["insights"])

if output["result_df"]:
    df = pd.DataFrame(output["result_df"]["rows"],
                      columns=output["result_df"]["columns"])
    print("\n=== Data Preview ===")
    print(df.head())


🤖 Model wants to call tool: execute_sql_query
📜 SQL generated: SELECT year, AVG(resale_price) as average_price FROM resale_data_2017_to_2025 GROUP BY year ORDER BY year;
✅ Query executed, got 9 rows
💡 Insights from the agent:
 Based on the data, I can see a clear trend in HDB resale prices over the years:

**Key Insights:**
- **2017-2019**: Prices remained relatively stable with a slight dip in 2019 (average $432,138)
- **2020**: Prices began to recover, reaching $452,279
- **2021-2025**: A significant upward trend emerged with consistent year-over-year growth
- **2021**: Major jump to $511,381 (13% increase from 2020)
- **2022**: Continued growth to $549,714
- **2023**: Reached $571,806
- **2024**: Surpassed $600,000 mark at $612,600
- **2025**: Highest average at $642,113

**Overall Trend**: HDB resale prices have increased by approximately **45%** from 2017 to 2025, with the most dramatic growth occurring in the post-2020 period. This suggests strong demand and potentially limited s